In [25]:
import pandas as pd
import wandb
from tqdm import tqdm

api = wandb.Api()
runs = api.runs("orientino/adam-sgd-gap")

data = "fineweb"
summary = []
for run in tqdm(runs):
    if run.config.get("data") != data:
        continue
    if run.config.get("wd") != 0.001:
        continue
    if run.config.get("n_layers") != 6:
        continue
    if run.tags:
        continue

    entry = {}
    entry["data"] = run.config.get("data")
    entry["lr"] = run.config.get("lr")
    entry["bs"] = run.config.get("bs")
    entry["mom"] = run.config.get("mom")
    entry["opt"] = run.config.get("opt")

    history = run.history(keys=["train/loss"], samples=10000)
    entry["tr_loss_history"] = history["train/loss"].tolist()
    history = run.history(keys=["val/loss"], samples=10000)
    entry["vl_loss_history"] = history["val/loss"].tolist()
    entry["tr_loss"] = run.summary.get("train/loss")
    entry["vl_loss"] = run.summary.get("val/loss")

    if data in ["c10", "c5m"]:
        history = run.history(keys=["val/acc1"], samples=10000)
        entry["vl_acc_history"] = history["val/acc1"].tolist()
        entry["vl_acc"] = run.summary.get("val/acc1")
    elif data in ["tinystories", "fineweb"]:
        history = run.history(keys=["val/ppl"], samples=10000)
        entry["vl_ppl_history"] = history["val/ppl"].tolist()
        entry["vl_ppl"] = run.summary.get("val/ppl")

    summary.append(entry)

len(summary)

100%|██████████| 2010/2010 [01:18<00:00, 25.46it/s] 


56

In [26]:
print(len(summary[0]["tr_loss_history"]))
# print(len(summary[0]["vl_loss_history"]))
# print(len(summary[0]["vl_acc_history"]))
grouped = (
    pd.DataFrame(summary)
    .groupby(["data", "opt", "bs"])
    .size()
    .reset_index(name="count")
)
print(grouped)

10000
      data   opt   bs  count
0  fineweb  adam  256     28
1  fineweb   sgd  256     28


In [27]:
filename = f"_results/gap_{data}_wd1e-3.csv"
df = pd.DataFrame(summary)
df.to_csv(filename, index=False)

In [ ]:
# df = pd.read_csv("_results/gap_vision_over_under.csv")
# df = df[df["data"] == "c10"]
# df = df[df["bs"] == 64]
# len(df["tr_loss_history"].iloc[0])

188513